## Question (1) a-e

In [1]:
from math import log
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
import random

In [2]:
iris = datasets.load_iris()
df=pd.DataFrame(iris.data)
target=pd.DataFrame(iris.target)
target=target.rename(columns={0:'target'})
df=df.rename(columns={0:'sepal_length',
  1:'sepal_width',
  2:'petal_length',
  3:'petal_width'})

In [3]:
df[:10]

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [4]:
target[:10]

,target
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [5]:
##decision_tree_code
def entropy(target):
    target=list(target)
    p_list=[]
    l=len(target)
    for i in list(set(target)):       #iterating through the unique class list in target
        p_list.append(target.count(i)/l)     #appending the probaility of each class to p_list
    if 1 in p_list:                          #if the probability of any class is 1 then entropy is zero
        return 0
    S=0                                 #S will store the entropy of the target
    for i in p_list:
        if i!=0:
            S+=i*log(i,2)
    return -S

def info_gain(examples,attributes,A,target):   #A is the attribute for which info gain is to be found
    S=entropy(target)
    l=len(target)
    sm=0
    tmp=np.array(examples)
    values_A=list(set(tmp[:,list(attributes).index(A)]))    #finding the unique values in attribute A
    for v in values_A:                                #find the information gain
        S_v=[]
        for i in range(len(target)):
            if examples[i][list(attributes).index(A)]==v:
                S_v.append(target[i])
        sm+=len(S_v)*entropy(S_v)/l
    i_g=S-sm
    return i_g

def findRoot(examples, target, attributes):
    ig_max=-1
    root=0
    for A in attributes:
        if info_gain(examples,attributes,A,target)>ig_max:
            ig_max=info_gain(examples, attributes, A, target)
            root=A
    return root

def DecisionTreeClassifier(examples, target, attributes):
    tree_={}
    attributes=list(attributes)
    if len(examples)==0:
        return {}
    if len(attributes)==0:      ###########improve here, assumed that target has only 2 classes: 0,1
        if sum(target)>=len(target)/2:
            return 1
        else:
            return 0
    if len(set(target))==1:
        tree_[attributes[0]]=target[0]
        return tree_
    root=findRoot(examples, target, attributes)
    examples=np.array(examples)
    values_root=list(set(examples[:,list(attributes).index(root)]))
    branch={}
    tree_[root]=branch    
    for v in values_root:
        branch[v]={}
        examp_new=[]
        target_new=[]    
        for i in range(len(target)):
            if examples[i][list(attributes).index(root)]==v:
                examp_new.append(examples[i])
                target_new.append(target[i])
        if examp_new==[]:
            if sum(target)>=len(target)/2:
                branch[v]=1
            else:
                branch[v]=0
        else:
            tmp=[z for z in attributes]
            tmp.remove(root)
            examp_new=np.delete(examp_new, np.s_[attributes.index(root):attributes.index(root)+1], axis=1)
            branch[v]=DecisionTreeClassifier(examp_new, target_new, tmp)
    attributes.remove(root)
    return tree_

def modify_examples(train_examples, split_values, attributes):
    train_examples_=[list(i) for i in train_examples]
    attributes=list(attributes)
    for i in range(len(train_examples)):
        for j in range(len(train_examples[0])):
            if train_examples[i][j]>=split_values[j]:
                train_examples_[i][j]=">" + str(split_values[j])
            else:
                train_examples_[i][j]="<"+str(split_values[j])

    return train_examples_

def modify_data(train_examples, train_target, attributes):
    change_point=[]
    for i in range(len(train_target)):
        try:
            if train_target[i]!=train_target[i+1]:
                change_point.append(i)
        except:
            pass
    tmp_data=[]
    for i in change_point:
        tmp_data.append((np.array(train_examples[i])+np.array(train_examples[i+1]))/2)
    tmp_data=np.array(tmp_data)
    tmp_data=np.around(tmp_data,decimals=3)
    split_values={}
    for i in range(len(attributes)):
        tmp_=tmp_data[:,i]
        tmp_.sort()
        tmp_=list(set(tmp_))
        mid=int(len(tmp_)/2)
        split_values[i]=tmp_[mid]
    #print(split_values)
    train_examples=modify_examples(train_examples, split_values,attributes)
    
    return [train_examples, split_values]


def randomforest(examples,target):
    features=[]
    attributes=list(examples.columns)
    [train_ex,split_values]=modify_data(np.array(examples),np.array(target),attributes)
    train_ex=pd.DataFrame(train_ex).rename(columns={0:'sepal_length',
  1:'sepal_width',
  2:'petal_length',
  3:'petal_width'})
    for _ in range(10):
        features.append(np.random.choice(np.array(attributes),3,replace=False))
    forest=[]
    for i in features:
        tmp=train_ex
        for j in attributes:
            if j not in i:
                tmp=tmp.drop(j,axis=1)
        forest.append(DecisionTreeClassifier(np.array(tmp),np.array(target),np.array(i)))
    return forest

In [6]:
forest=randomforest(df,np.transpose(np.array(target))[0])

In [7]:
k=modify_data(np.array(df),np.transpose(np.array(target))[0], df.columns)

In [8]:
k[0]

[['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '<3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '<3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '<3.05', '<5.05', '<1.9'],
 ['<6.0', '<3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 ['<6.0', '<3.05', '<5.05', '<1.9'],
 ['<6.0', '>3.05', '<5.05', '<1.9'],
 

In [9]:
DecisionTreeClassifier(k[0],np.transpose(np.array(target))[0], list(df.columns))

{'petal_length': {'<5.05': {'sepal_width': {'<3.05': {'petal_width': {'<1.9': {'sepal_length': {'<6.0': 1,
        '>6.0': 1}},
      '>1.9': {'sepal_length': 2}}},
    '>3.05': {'sepal_length': {'<6.0': {'petal_width': {'<1.9': 0}},
      '>6.0': {'petal_width': 1}}}}},
  '>5.05': {'petal_width': {'<1.9': {'sepal_width': {'<3.05': {'sepal_length': {'<6.0': 1,
        '>6.0': 1}},
      '>3.05': {'sepal_length': 2}}},
    '>1.9': {'sepal_length': 2}}}}}

In [10]:
forest

[{'sepal_length': {'<1.9': {'petal_width': {'<6.0': {'sepal_width': {'<3.05': 1,
       '>3.05': 0}},
     '>6.0': {'sepal_width': {'<3.05': 1, '>3.05': 1}}}},
   '>1.9': {'petal_width': 2}}},
 {'sepal_width': {'<1.9': {'sepal_length': {'<6.0': {'petal_width': {'<3.05': 1,
       '>3.05': 0}},
     '>6.0': {'petal_width': {'<3.05': 1, '>3.05': 1}}}},
   '>1.9': {'sepal_length': 2}}},
 {'petal_length': {'<5.05': {'sepal_width': {'<3.05': {'petal_width': {'<1.9': 1,
       '>1.9': 1}},
     '>3.05': {'petal_width': {'<1.9': 0}}}},
   '>5.05': {'petal_width': {'<1.9': {'sepal_width': {'<3.05': 1, '>3.05': 1}},
     '>1.9': {'sepal_width': 2}}}}},
 {'petal_width': {'<5.05': {'sepal_length': {'<6.0': {'petal_length': {'<1.9': 0,
       '>1.9': 1}},
     '>6.0': {'petal_length': {'<1.9': 1, '>1.9': 1}}}},
   '>5.05': {'petal_length': {'<1.9': {'sepal_length': {'<6.0': 1, '>6.0': 1}},
     '>1.9': {'sepal_length': 2}}}}},
 {'sepal_width': {'<1.9': {'petal_width': {'<6.0': {'sepal_length': {'<